In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s3e6/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e6/test.csv')
train_add = pd.read_csv('/kaggle/input/paris-housing-price-prediction/ParisHousing.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
print(sum(train_add.isnull().sum())==0)
print(sum(train.isnull().sum())==0)
print(sum(test.isnull().sum())==0)

In [ ]:
train.describe()

In [ ]:
for i, val in enumerate(train['made']):
    if val>2022:
        train.loc[i,'made'] = int(np.mean(train['made']))
for i, val in enumerate(train['floors']):
    if val>100:
        train.loc[i,'floors'] = int(np.mean(train['floors']))  

cat_cols=[]
for i, col in enumerate(train.columns):
    if col!='id':
        num = len(train[col].value_counts().index)
        if num <10: 
            print(train[col].value_counts())
            cat_cols.append(col)
            
for col in ['numPrevOwners','made','floors','numberOfRooms','cityPartRange']:
    cat_cols.append(col)
            
for col in cat_cols:
    train[col] = train[col].astype('category')
    train_add[col] = train_add[col].astype('category')
    test[col] = test[col].astype('category')
    
    
test_ids = test['id']
test.drop(['id'],axis=1,inplace=True)
train.drop(['id'],axis=1,inplace=True)


train = train[train.columns]
train_add = train_add[train.columns]    

train.info()

In [ ]:
fig1, axis1 = plt.subplots(5,2,figsize=(15,30))
j=0
p=0
for i, col in enumerate(cat_cols):
    if p>=2:
        j+=1
        p-=2
        if len(train[col].value_counts().index) >10:
            sns.histplot(data=train,x=col,ax=axis1[j,p],label='col',bins=10)
        else:
            sns.countplot(data=train,x=col,ax=axis1[j,p],label='col')
        p+=1
    elif p<2:
        if len(train[col].value_counts().index) >10:
            sns.histplot(data=train,x=col,ax=axis1[j,p],label='col',bins=10)
        else:
            sns.countplot(data=train,x=col,ax=axis1[j,p],label='col')
        p+=1

fig2, axis2 = plt.subplots(7,1, figsize=(30,30)) 
j=0
p=0
for i, col in enumerate([col for col in train.columns if col not in cat_cols]):
    if p>=1:
        j+=1
        p-=1
        sns.kdeplot(data=train,x=col,ax=axis2[j])
        p+=1
    elif p<1:
        sns.kdeplot(data=train,x=col,ax=axis2[j])
        p+=1

        
    
        

In [ ]:
import sklearn
import tensorflow as tf
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from category_encoders import *
import catboost as cat
import lightgbm as lgb
import xgboost as xgb
import optuna


In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(train.iloc[:,:-1],train.iloc[:,-1], test_size=0.2,random_state=1, shuffle=True)
train_encoder = JamesSteinEncoder()
X_train[test.columns] = train_encoder.fit_transform(X_train,y_train)
X_cv[test.columns]  = train_encoder.transform(X_cv)
test[test.columns]  = train_encoder.transform(test)

#train_add_encoder = JamesSteinEncoder()
#train_add[test.columns] = train_add_encoder.fit_transform(train_add.iloc[:,:-1],train_add.iloc[:,-1])

#X_train = pd.concat([X_train,train_add.iloc[:,:-1]],axis=0)
#y_train = pd.concat([y_train,train_add.iloc[:,-1]],axis=0)
 

In [ ]:
model = xgb.XGBRegressor(random_state=1,n_estimators=1000,learning_rate=0.1)

params_grid = {'max_depth':range(2,11),
              'min_child_weight':range(2,10,2)}

gscv = GridSearchCV(estimator=model, param_grid=params_grid,cv=5,scoring='neg_mean_squared_error',n_jobs=-1,verbose=1)
gscv.fit(X_train,y_train)
print(np.sqrt(-gscv.best_score_))
print(gscv.best_params_)

In [ ]:
dep = gscv.best_params_['max_depth']
mcw = gscv.best_params_['min_child_weight']

In [ ]:
model = xgb.XGBRegressor(random_state=1,n_estimators=1000,learning_rate=0.1,max_depth=dep,min_child_weight=mcw)

params_grid = {'gamma':[i/10 for i in range(0,11)]}

gscv = GridSearchCV(estimator=model, param_grid=params_grid,cv=5,scoring='neg_mean_squared_error',n_jobs=-1,verbose=1)
gscv.fit(X_train,y_train)
print(np.sqrt(-gscv.best_score_))
print(gscv.best_params_)

In [ ]:
gam = gscv.best_params_['gamma']

In [ ]:
model = xgb.XGBRegressor(random_state=1,n_estimators=1000,learning_rate=0.1,max_depth=dep,min_child_weight=mcw,gamma=gam)

params_grid = {'colsample_bytree':[i/100 for i in range(50,100,5)],
              'subsample':[i/100 for i in range(50,100,5)]}

gscv = GridSearchCV(estimator=model, param_grid=params_grid,cv=5,scoring='neg_mean_squared_error',n_jobs=-1,verbose=1)
gscv.fit(X_train,y_train)
print(np.sqrt(-gscv.best_score_))
print(gscv.best_params_)

In [ ]:
col = gscv.best_params_['colsample_bytree']
sub = gscv.best_params_['subsample']

In [ ]:
model = xgb.XGBRegressor(random_state=1,n_estimators=1000,learning_rate=0.1,max_depth=dep,min_child_weight=mcw,gamma=gam,colsample_bytree=col,subsample=sub)

params_grid = {'reg_alpha':[1e-06,1e-05,1e-4,1e-03,1e-2,1e-01,1,10,100],
              'reg_lambda':[1e-06,1e-05,1e-4,1e-03,1e-2,1e-01,1,10,100]}

gscv = GridSearchCV(estimator=model, param_grid=params_grid,cv=5,scoring='neg_mean_squared_error',n_jobs=-1,verbose=1)
gscv.fit(X_train,y_train)
print(np.sqrt(-gscv.best_score_))
print(gscv.best_params_)

#baseline model
model = cat.CatBoostRegressor(random_state=1,verbose=0,learning_rate= 0.01162479640254175, reg_lambda= 1.1363014852826037, bootstrap_type= 'Bayesian', random_strength =44.81785544243456, depth= 16, grow_policy= 'Lossguide', min_data_in_leaf= 694, max_bin=962, colsample_bylevel= 0.9997990376682409, bagging_temperature= 36.359600424536026, max_leaves= 21)
model.fit(X_train,y_train,eval_set=(X_cv,y_cv),use_best_model=True,early_stopping_rounds=200)

model1 = cat.CatBoostRegressor(random_state=1,verbose=0)
model1.fit(X_train,y_train,eval_set=(X_cv,y_cv),use_best_model=True,early_stopping_rounds=200)

pred = (model.predict(X_cv)+model1.predict(X_cv))*0.5
mean_squared_error(y_cv,pred,squared=False)

submission = pd.DataFrame({'id':test_ids,'price':((model.predict(test)+model1.predict(test))*0.5).reshape(-1,)})
submission.to_csv('submission.csv',index=False)
#'learning_rate': 0.01162479640254175, 'reg_lambda': 1.1363014852826037, 'bootstrap_type': 'Bayesian', 'random_strength': 44.81785544243456, 'depth': 16, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 694, 'max_bin': 962, 'colsample_bylevel': 0.9997990376682409, 'bagging_temperature': 36.359600424536026, 'max_leaves': 21

n_fold = KFold(3)
def objective(trial):
    score_list=[]
    params ={'verbose':0,
             'eval_metric':'RMSE',
             'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.8),
             'reg_lambda':trial.suggest_loguniform('reg_lambda',1,1000),
             'bootstrap_type':trial.suggest_categorical('bootstrap_type',['Bayesian','Bernoulli','MVS']),
             'random_strength':trial.suggest_float('random_strength',1,1000),
             'depth':trial.suggest_int('depth',5,16),
             'grow_policy':trial.suggest_categorical('grow_policy',['Depthwise','Lossguide']),
             'min_data_in_leaf':trial.suggest_int('min_data_in_leaf',1,1000),
             'max_bin':trial.suggest_int('max_bin',1,1000),
             'colsample_bylevel':trial.suggest_float('colsample_bylevel',0.,1)
             
    }
    
    if params['bootstrap_type']=='Bayesian':
        params['bagging_temperature']=trial.suggest_float('bagging_temperature',1,100)
    if params['bootstrap_type'] =='Bernoulli' :
        params['subsample']=trial.suggest_float('subsample',0.,1)
    if params['grow_policy'] == 'Lossguide':
        params['max_leaves']=trial.suggest_int('max_leaves',10,100)
    if params['bootstrap_type']=='MVS':
        params['mvs_reg']:trial.suggest_loguniform('mvs_reg',1e-05,1000)
        params['subsample']=trial.suggest_float('subsample',0.,1)
            
    for i, (train_ids,cv_ids) in enumerate(n_fold.split(X_train,y_train)):
        X_tr = X_train.iloc[train_ids]
        X_c = X_train.iloc[cv_ids]
        y_tr = y_train.iloc[train_ids]
        y_c = y_train.iloc[cv_ids]
        
        #ncoder = CatBoostEncoder()
        #_tr[X_tr.columns] = encoder.fit_transform(X_tr,y_tr)
        #_c = encoder.transform(X_c,y_c)
    
    model1= cat.CatBoostRegressor(**params)
    model1.fit(X_tr,y_tr,eval_set=(X_c,y_c),use_best_model=True,early_stopping_rounds=200)
    score = mean_squared_error(y_c,model1.predict(X_c),squared=False)
    score_list.append(score)
    return sum(score_list)/len(score_list)

study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials=500)